In [1]:
import json
import pandas as pd
import numpy as np
import plotly as py

select reveiws in brunch/breakfast restaurants

In [ ]:
df1 = pd.read_json("review_train.json",orient='records',lines = True)

In [56]:
df2=pd.read_json("business_train.json",orient='records',lines=True)

In [57]:
ind=[]
cate=df2["categories"].tolist()
for i in range(len(cate)):
    if cate[i]==None:
        continue
    if ("Chinese" in cate[i])|("Thai" in cate[i])|("Japanese" in cate[i])|("Asian" in cate[i])|("Korean" in cate[i]):
        continue
    if ("Brunch" in cate[i])|("Breakfast" in cate[i]):
        ind.append(i)

In [58]:
dfbrunch=df2.iloc[ind]
#sum(dfbrunch["text_c"]) #how many reviews
id_extract=dfbrunch["business_id"].tolist()

In [ ]:
df1=df1.loc[df1["business_id"].isin(id_extract)]

In [24]:
brunch_star = df1.stars

In [25]:
brunch_temp = df1[:]

transform emoticons into English words (the collection of emoticons can see in wikipedia: List of emoticons)

In [26]:
import re
def convert_emo(text):
    happy = r"(:-\)|:\)|:-\]|:\]|:3|:c\)|:>|8\)|8-\)|:o\)|=\)|:\}|:\^\)|:-\)\))" 
    laugh = r"(:‑D|:D|8‑D|8D|x‑D|xD|X‑D|XD|=‑D|=D|=‑3|=3|B\^D)" 
    sad = r"(:‑\(|:\(|:‑c|:c|:‑<|:<|:‑\[|:\[|:\{|;\(|>:\[)" 
    angry = r"(:-\|\||:@|>:\()" 
    cry = r"(:'‑\(|:'\()" 
    horror = r"(D:<|D:|D8|D;|D=|DX|D‑':|v\.v)" 
    surprise = r"(>:O|:‑O|:O|:‑o|:o|8‑0|O_O|o‑o|O_o|o_O|o_o|O-O)" 
    kiss = r"(:\*|:-\*|:\^\*|\( '\}\{'\))" 
    wink = r"(;‑\)|;\)|\*-\)|\*\)|;‑\]|;\]|;D|;\^\)|:‑,)" 
    skeptical = r"(>:\\\\|>:/|:‑/|:‑\.|:/|:\\\\|=/|=\\\\|:L|=L|:S|>\.<)" 
    text = re.sub(happy,' happy ',text)
    text = re.sub(laugh,' laugh ',text)
    text = re.sub(sad,' sad ',text)
    text = re.sub(angry,' angry ',text)
    text = re.sub(cry,' cry ',text)
    text = re.sub(horror,' horror ',text)
    text = re.sub(surprise,' surprise ',text)
    text = re.sub(kiss,' kiss ',text)
    text = re.sub(wink,' wink ',text)
    text = re.sub(skeptical,' skeptical ',text)
    return text

In [27]:
brunch_temp.text = brunch_temp.text.apply(convert_emo)

remove '\n' in reviews

In [28]:
def remove_gangn(text):
    text = re.sub('\n',' ',text)
    return text

In [29]:
brunch_temp.text = brunch_temp.text.apply(remove_gangn)

remove underline in reviews and convert into lowercase

In [30]:
def rm_underline(text):
    text = re.sub('_',' ',text.lower())
    return text

In [31]:
brunch_temp.text = brunch_temp.text.apply(rm_underline)

expand abbreviation

In [32]:
def convert_abb(text):
    text = re.sub('n\'t',' not',text)
    text = re.sub('\'m',' am',text)
    text = re.sub('\'s', ' is',text)
    text = re.sub('\'ve',' have',text)
    text = re.sub('\'d',' would',text)
    return text

In [33]:
brunch_temp.text = brunch_temp.text.apply(convert_abb)

add not_ to every word between negation and following punctuation

In [34]:
import re
def add_negation(text):
    negation = ['not', 'never', 'no']
    words = text.split()[:]
    flag = 0
    for index, word in enumerate(words):
        if(flag != 0):
            words[index] = 'not_' + words[index]
        if(re.match('.*[;,.?!\'\"]$', word)):
            flag = 0
        if(word in negation):
            flag = 1
    text = " ".join(words)
    return(text)

In [35]:
brunch_temp.text = brunch_temp.text.apply(add_negation)

remove punctuation now

In [36]:
def remove_punc(text):
    text = re.sub('[/*-?!:;,\.\'\"\)\(]', ' ', text)
    return text

In [37]:
brunch_temp.text = brunch_temp.text.apply(remove_punc)

In [38]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#nltk.download('averaged_perceptron_tagger')

lemmatizer

In [39]:
def lemmatizer_all(text):
    new_text = []
    lemmatizer = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(text)):
        if tag.startswith('NN'):
            new_text.append(lemmatizer.lemmatize(word, pos='n'))
        elif tag.startswith('VB'):
            new_text.append(lemmatizer.lemmatize(word, pos='v'))
        elif tag.startswith('JJ'):
            new_text.append(lemmatizer.lemmatize(word, pos='a'))
        elif tag.startswith('R'):
            new_text.append(lemmatizer.lemmatize(word, pos='r'))
        else:
            new_text.append(word)
    return ' '.join(new_text)

In [40]:
brunch_temp.text = brunch_temp.text.apply(lemmatizer_all)

In [48]:
len(brunch_temp)

505696

In [54]:
brunch_temp.index = list(range(len(brunch_temp)))

remove no_important_word and words with no importance part of speech

In [56]:
no_importance_pos = ['IN','MD','PRP','PRP$','TO','WDT','WP','WP$','WRB']
no_importance_word = ['i','he','she','they','the','a','an','this','that','those','be','not_the','there','not_to','not_a','not_me','not_my',
                      'not_in','not_of','not_or','not_is','not_was','not_for','not_not_be','do_not','then','and','not','always','go','have',
                      'do','get','so','one','just','all','not_and','so', 'say','here','ever','not_get','not_i','not_that','not_this',
                      'not_not','guy','some','call','guy','re','one','not_not_even','then','and','not','always','go','have','do','get',
                      'so','one','just','all','not_and', 'say','here','ever','not_get','not_i','not_that','not_this','not_not','guy','some',
                      'guy','re','one', 'not_have','not_be','not_','not_it','not_be','not_you','not_on','even','not_you',
                      'not_on','up','ll','now','too','need', 'really','now','two']
only_text = []
index = []
stars = []
for i in range(len(brunch_temp)):
    try:
        only_text.append(' '.join(brunch_temp.text.loc[i].split(' ')))
        stars.append(brunch_star[i])
    except:
        index.append(i)
        pass

In [57]:
len(only_text)

505696

In [58]:
new_text = []
star = []
index2 = []
for i in range(len(only_text)):
    try:
        retain = []
        POS = pos_tag(only_text[i].split(' '))
        for word in POS:
            if word[1] not in no_importance_pos and word[0] not in no_importance_word and len(word[0])!= 1:
                retain.append(word[0])
        new_text.append(' '.join(retain))
        star.append(stars[i])
    except:
        index2.append(i)
        pass

In [62]:
from gensim.models.phrases import Phrases, Phraser

D:\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning:

detected Windows; aliasing chunkize to chunkize_serial



extract phrases, make phrases into one word

In [63]:
only_word = [sentence.split(' ') for sentence in new_text]

In [67]:
ph = Phraser(Phrases(only_word))

In [68]:
final_text = []
for i in range(len(only_word)):
    final_text.append(' '.join(ph[only_word[i]]))

In [69]:
final_text

['walk friday_afternoon sit table bar walk min or not_even_not_think not_they not_realized_not_we not_walked however everyone bar notice walk service non_existent best not_good_not_way not_new_not_business not_start oh well location different thing past several_year add list smdh',
 'pick any meat planet chef make mexican_style dish amazing flavor wow fish lamb taco die drink great well',
 'flat out fantastic twice croissant bomb not_wait_not_until not_can not_try_not_other not_items new breakfast spot ahwahtukee service great well family_run business come support',
 'employee busy chat each_other employee head drive_thru window talk people entire time cook come outside make food hand make food',
 'yum kale grit definitely again menu choice unique farm_fresh delicious',
 'wonderful experience sit_patio watch food channel listen_music menu great conversation out country guests patio good place kid patio drown_out loud child food great bite spot only wait_minute table peak dinner hour se

In [71]:
brunch_star.index = list(range(len(brunch_star)))

In [72]:
brunch_star

0         1
1         5
2         5
3         2
4         5
5         4
6         4
7         5
8         1
9         3
10        4
11        4
12        4
13        5
14        4
15        4
16        1
17        4
18        5
19        4
20        1
21        5
22        5
23        2
24        2
25        4
26        5
27        5
28        3
29        5
         ..
505666    1
505667    4
505668    5
505669    3
505670    5
505671    4
505672    2
505673    3
505674    2
505675    5
505676    1
505677    5
505678    5
505679    4
505680    5
505681    5
505682    5
505683    5
505684    5
505685    5
505686    1
505687    5
505688    5
505689    5
505690    5
505691    5
505692    5
505693    5
505694    5
505695    3
Name: stars, Length: 505696, dtype: int64

In [77]:
data = [list(brunch_star),final_text]

In [78]:
data = pd.DataFrame(data)
data = data.T
data.rename(columns = {0:'stars',1:'text'},inplace = True)

In [79]:
data.to_csv('brunch_processed.csv')